# Introduction

This tutorial demonstrates density based clustering of crystal orientations with and without the application of symmetry using simulated data.

This functionaility has been checked to run in orix-0.2.0 (December 2019). Bugs are always possible, do not trust the code blindly, and if you experience any issues please report them here: https://github.com/pyxem/orix-demos/issues

# Contents

1. <a href='#gen'> Generate artificial data</a>
2. <a href='#clu'> Misorientation clustering</a>
3. <a href='#vis'> Visualisation</a>

Import orix classes and various dependencies

In [ ]:
%matplotlib qt

# Import core external
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN

# Import orix classes 
from orix.quaternion.orientation import Orientation, Misorientation
from orix.quaternion.rotation import Rotation
from orix.quaternion.symmetry import Oh
from orix.quaternion.orientation_region import OrientationRegion
from orix.vector.neo_euler import AxAngle
from orix import plot

# Colorisation & Animation
from skimage.color import label2rgb
from matplotlib.colors import to_rgb, to_hex
import matplotlib.animation as animation

plt.rc('font', size=6)

# <a id='gen'></a> 1. Generate artificial data

Generate 3 random vonMises distributions as model clusters and set Oh symmetry

In [ ]:
# Set up data
d1 = Orientation.random_vonmises(50, alpha=50)
d2_0 = Rotation.from_neo_euler(AxAngle.from_axes_angles((1, 0, 0), np.pi/4))
d2 = Orientation.random_vonmises(50, alpha=50, reference=d2_0)
d3_0 = Rotation.from_neo_euler(AxAngle.from_axes_angles((1, 1, 0), np.pi/3))
d3 = Orientation.random_vonmises(50, alpha=50, reference=d3_0)
dat = Orientation.stack([d1, d2, d3]).flatten().set_symmetry(Oh)

# <a id='clu'></a> 2. Orientation clustering

## 2.1 Perform clustering without application of crystal symmetry

Compute misorientations, i.e. distance between orientations

In [ ]:
D = (~dat).outer(dat).angle.data

Perform clustering

In [ ]:
dbscan_naive = DBSCAN(0.3, 10, metric='precomputed').fit(D)
print('Labels:', np.unique(dbscan_naive.labels_))

## 2.2 Perform clustering with application of crystal symmetry

Compute misorientations, i.e. distance between orientations, with symmetry

In [ ]:
D = Misorientation((~dat).outer(dat)).set_symmetry(Oh,Oh).angle.data

Perform clustering

In [ ]:
dbscan = DBSCAN(0.3, 20, metric='precomputed').fit(D)
print('Labels:', np.unique(dbscan.labels_))

This should have shown that without symmetry there are 6 clusters, whereas with symmetry there are 3.

# <a id='vis'></a> 1. Visualisation

Assign colours to each cluster

In [ ]:
colors = [to_rgb('C{}'.format(i)) for i in range(10)]  # ['C0', 'C1', ...]
c = label2rgb(dbscan.labels_, colors=colors)
c_naive = label2rgb(dbscan_naive.labels_, colors=colors)

Specify fundamental zone based on symmetry

In [ ]:
fr = OrientationRegion.from_symmetry(Oh)

Generate plot

In [ ]:
# Figure
fig = plt.figure(figsize=(12, 7))

# This is the left hand plot
ax1 = fig.add_subplot(121, projection='axangle', aspect='equal')
ax1.scatter(dat, c=c_naive)
ax1.plot_wireframe(fr, color='gray', alpha=0.5, linewidth=0.5, rcount=30, ccount=30)
ax1.set_axis_off()

ax1.set_xlim(-0.8, 0.8)
ax1.set_ylim(-0.8, 0.8)
ax1.set_zlim(-0.8, 0.8)

# This is the right hand plot
ax2 = fig.add_subplot(122, projection='axangle', aspect='equal')
ax2.scatter(dat, c=c)
ax2.plot_wireframe(fr, color='gray', alpha=0.5, linewidth=0.5, rcount=30, ccount=30)
ax2.set_axis_off()

ax2.set_xlim(-0.8, 0.8)
ax2.set_ylim(-0.8, 0.8)
ax2.set_zlim(-0.8, 0.8)

plt.tight_layout()

Generate an animation of the plot

In [ ]:
def animate(angle):
    ax1.view_init(15, angle)
    ax2.view_init(15, angle)
    plt.draw()

ani = animation.FuncAnimation(fig, animate, np.linspace(75, 360+74, 720), interval=25)
plt.show()